In [0]:
from pyspark.sql.functions import current_timestamp
def add_ingestion_date(input_df):
  return input_df.withColumn('ingestion_date', current_timestamp())

In [0]:
def arrange_partition_column(input_df, partition_column):
    col_list = []
    for col in  input_df.schema.names:
        if col != partition_column:
            col_list.append(col)
    col_list.append(partition_column)
    return input_df.select(col_list)

In [0]:
def overwrite_partition(input_df, partition_column, table_name, db_name ):
    output_df = arrange_partition_column(input_df, partition_column)

    spark.conf.set('spark.sql.sources.partitionOverwriteMode','dynamic')

    if(spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):
       output_df.write.mode('overwrite').insertInto(f"{db_name}.{table_name}")
    else:
       output_df.write.mode('overwrite').partitionBy(partition_column).format('parquet').saveAsTable(f"{db_name}.{table_name}")

In [0]:
def df_column_to_list(input_df, column_name):
    df_row_list = input_df.select(column_name).distinct().collect()
    column_list = [row[column_name] for row in df_row_list]
    return column_list